# Final project: Time-series data and application to stock markets

Part 2: Vietnam Stock Prediction 

In [15]:
# Load libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import os
from tensorflow import keras
from sklearn.model_selection import TimeSeriesSplit, train_test_split


In [5]:
# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [33]:
companies_overview = pd.read_csv("data-vn-20230228/ticker-overview.csv")
company_names = pd.read_csv("data-vn-20230228/companies.csv")

In [34]:
#Step 1: Filter companies in technology sector 
tech_companies = companies_overview[companies_overview["industryEn"] == "Technology"]['ticker'].to_list()
len(tech_companies)

30

In [35]:
#Get group codes for these companies -> validate data points number
root_path = 'data-vn-20230228/'
history_path = root_path + 'stock-historical-data/'
dfs = []
saved_companies = []
for company in tech_companies: 
    group_code = str(company_names[company_names.ticker == company]['group_code'].values[0])
    filename = history_path + company + "-" + group_code + '-History.csv'
    df = pd.read_csv(filename)
    if len(df) >= 1200: 
        dfs.append(df)
        saved_companies.append(company)

In [36]:
len(dfs)

28

In [10]:
#Save constants for program
window_size = 120
pred_time = 7
features = ["Open", "High", "Low", "Close"]


In [81]:
# Split the dataset into time windows to get data samples.

def get_samples(df,window_size, target_pred=1): 
    X_data = []
    y_data = []
    i = 0 
    while i < (len(df) - window_size - target_pred):
        X_data.append(df.iloc[i:i + window_size,:].to_numpy())
        y_data.append(df.iloc[i + window_size:i + window_size + target_pred, :].to_numpy())
        i += target_pred
    return X_data, y_data

def preprocessing(X_data, y_data, window_size):
    # Import the library
    from sklearn.model_selection import train_test_split

    # Split data into train, val and test. 
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

    X_train = np.array(X_train)
    X_val = np.array(X_val)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    y_test = np.array(y_test)
    
    #Print shape of the training, validation and test set
    print("Shape of training set: ", X_train.shape, "label shape:", y_train.shape)
    print("Shape of validation set: ", X_val.shape, "label shape", y_val.shape)
    print("Shape of test set: ", X_test.shape, "label shape:", y_test.shape)
    return X_train, X_test, X_val, y_train, y_test, y_val

def norm_data(X, y):
    X_norm = X.copy()
    y_norm = y.copy()
    for i in range(0, len(X)):
        min_feature = np.min(X[i])
        max_feature = np.max(X[i])
        X_norm[i] = (X[i] - min_feature) / (max_feature - min_feature)
        y_norm[i] = (y[i] - min_feature) / (max_feature - min_feature)
    return X_norm, y_norm


In [79]:
#Select previous model
def train_model(X_train_norm, y_train_norm, window_size, pred_time): 
    from keras.layers import LSTM, Flatten, Dense, Cropping1D, Input
    model = tf.keras.Sequential()
    model.add(Input(shape=(window_size, len(features))))
    model.add(LSTM(units=128, return_sequences=True)) # Output dimension of LSTM layer
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(Dense(1))
    model.add(Cropping1D(cropping=(window_size - pred_time, 0)))
    # Compile the model
    model.compile(optimizer= tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), loss='mse', metrics=['mse'])
    # Train the model
    model.fit(X_train_norm, y_train_norm, epochs=15, verbose=0)
    return model

In [75]:
good_companies = []
# !mkdir -p "saved_model/vnstock/"
save_path = "saved_model/vnstock/"

def train_and_select_company(company, df): 
    window_size = 120
    pred_time = 7
    X, y = get_samples(df, window_size, target_pred = pred_time)
    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=0)
    X_train_norm, y_train_norm = norm_data(X_train, y_train)
    X_test_norm, y_test_norm = norm_data(X_test, y_test)
    mses = []
    models = []

    for n, (train_index, test_index) in enumerate(TimeSeriesSplit(n_splits=5).split(X_train, y_train)):
        print("On fold: --", n)
#         print("train_index", train_index , "test_index", test_index)
        X_val_fold, y_val_fold = X_train.copy()[test_index], y_train.copy()[test_index]
        X_train_fold, y_train_fold = X_train.copy()[train_index], y_train.copy()[train_index]
        X_train_fold_norm, y_train_fold_norm = norm_data(X_train_fold, y_train_fold)
        X_val_fold_norm, y_val_fold_norm = norm_data(X_val_fold, y_val_fold)
        # Create and train the model
        model = train_model(X_train_fold_norm, y_train_fold_norm, window_size, pred_time)
        val_mse = model.evaluate(X_val_fold_norm, y_val_fold_norm, verbose=0)[1]
        test_mse = model.evaluate(X_test_norm, y_test_norm, verbose=0)[1]
        print("Val MSE:", val_mse, "- Test MSE:", test_mse)
        mses.append(test_mse)
        models.append(model)
    overall_mse = np.average(mses)
    print("Overall MSE: ", overall_mse)
    #Save best model to predict
    if overall_mse < 0.07: 
        model = models[np.argmin(mses)].save(filepath=save_path + company, include_optimizer=False) 
        good_companies.append(company)
        

## Basic Questions: 

How to decide which company to keep and which to sell? 
-> Based on price and indicators -> Common indicators for technical analysis are Relative strength index (RSI) and Supertrend Indicator (SI)
-> use pandas-ta, a popular library to calculate such technical scores for this data set

In [114]:
def calc_indicator(df):
    """Output valuable indicators for user financial decision"""
    indicator = {"RSI": ta.rsi(df["Close"], length=3)}
    return indicator

To predict price for both open, close, high and low, I used a model taking input of such features in 120 days time window to predict for the next 7 days. Save models for companies with cross-validation MSE lower than 0.07 threshold. 

In [82]:
#Start predict prices for all 28 companies in the list
for i in range(len(saved_companies)): 
    company_name = saved_companies[i]
    print("Train and select model for", company_name)
    df = dfs[i][features]
    train_and_select_company(company_name, df)
    

Train and select model for CKV
On fold: -- 0
Val MSE: 0.05839037895202637 - Test MSE: 0.11583037674427032
On fold: -- 1
Val MSE: 0.028916919603943825 - Test MSE: 0.09119730442762375
On fold: -- 2
Val MSE: 0.04061967134475708 - Test MSE: 0.0820302739739418
On fold: -- 3
Val MSE: 0.044531047344207764 - Test MSE: 0.08165270835161209
On fold: -- 4
Val MSE: 0.05174537003040314 - Test MSE: 0.0861135795712471
Overall MSE:  0.09136484861373902
Train and select model for CMG
On fold: -- 0
Val MSE: 0.06544730067253113 - Test MSE: 0.11394822597503662
On fold: -- 1
Val MSE: 0.07485418766736984 - Test MSE: 0.05871550366282463
On fold: -- 2
Val MSE: 0.03654702007770538 - Test MSE: 0.045640140771865845
On fold: -- 3
Val MSE: 0.056075625121593475 - Test MSE: 0.04443804547190666
On fold: -- 4
Val MSE: 0.029322944581508636 - Test MSE: 0.04372850060462952
Overall MSE:  0.06129408329725265


INFO:tensorflow:Assets written to: saved_model/vnstock/CMG/assets


INFO:tensorflow:Assets written to: saved_model/vnstock/CMG/assets


Train and select model for CMT
On fold: -- 0
Val MSE: 0.1233333870768547 - Test MSE: 0.2051384598016739
On fold: -- 1
Val MSE: 0.053562525659799576 - Test MSE: 0.05038287863135338
On fold: -- 2
Val MSE: 0.04482569918036461 - Test MSE: 0.05563989281654358
On fold: -- 3
Val MSE: 0.03275752440094948 - Test MSE: 0.04720687493681908
On fold: -- 4
Val MSE: 0.037249840795993805 - Test MSE: 0.0540614053606987
Overall MSE:  0.08248590230941773
Train and select model for ELC
On fold: -- 0
Val MSE: 0.05171627551317215 - Test MSE: 0.04268678277730942
On fold: -- 1
Val MSE: 0.056750524789094925 - Test MSE: 0.046980537474155426
On fold: -- 2
Val MSE: 0.031056858599185944 - Test MSE: 0.04631996899843216
On fold: -- 3
Val MSE: 0.02189546637237072 - Test MSE: 0.04393366351723671
On fold: -- 4
Val MSE: 0.04437632858753204 - Test MSE: 0.047528933733701706
Overall MSE:  0.045489977300167087


INFO:tensorflow:Assets written to: saved_model/vnstock/ELC/assets


...

/var/folders/gk/fvlxt4n12ldf5rf0ycgz750h0000gn/T/ipykernel_837/2445643204.py:40: RuntimeWarning: invalid value encountered in divide
  X_norm[i] = (X[i] - min_feature) / (max_feature - min_feature)
/var/folders/gk/fvlxt4n12ldf5rf0ycgz750h0000gn/T/ipykernel_837/2445643204.py:41: RuntimeWarning: invalid value encountered in divide
  y_norm[i] = (y[i] - min_feature) / (max_feature - min_feature)
/var/folders/gk/fvlxt4n12ldf5rf0ycgz750h0000gn/T/ipykernel_837/2445643204.py:41: RuntimeWarning: divide by zero encountered in divide
  y_norm[i] = (y[i] - min_feature) / (max_feature - min_feature)


Val MSE: nan - Test MSE: 0.3111576437950134
On fold: -- 1
Val MSE: nan - Test MSE: nan
On fold: -- 2
Val MSE: nan - Test MSE: nan
On fold: -- 3
Val MSE: nan - Test MSE: nan
On fold: -- 4
Val MSE: nan - Test MSE: nan
Overall MSE:  nan
Train and select model for VLA
On fold: -- 0
Val MSE: 0.053113292902708054 - Test MSE: 0.08269046992063522
On fold: -- 1
Val MSE: 0.03688882663846016 - Test MSE: 0.061779752373695374
On fold: -- 2
Val MSE: 0.02372221276164055 - Test MSE: 0.0653274729847908
On fold: -- 3
Val MSE: 0.04477129504084587 - Test MSE: 0.06657061725854874
On fold: -- 4
Val MSE: 0.12331400066614151 - Test MSE: 0.06307578831911087
Overall MSE:  0.0678888201713562


INFO:tensorflow:Assets written to: saved_model/vnstock/VLA/assets


INFO:tensorflow:Assets written to: saved_model/vnstock/VLA/assets


Train and select model for VTC
On fold: -- 0
Val MSE: 0.04057393595576286 - Test MSE: 0.08662206679582596
On fold: -- 1
Val MSE: 0.04010061174631119 - Test MSE: 0.09229376912117004
On fold: -- 2
Val MSE: 0.08069667220115662 - Test MSE: 0.0870145782828331
On fold: -- 3
Val MSE: 0.03679899871349335 - Test MSE: 0.08962731808423996
On fold: -- 4
Val MSE: 0.07185742259025574 - Test MSE: 0.08240296691656113
Overall MSE:  0.08759213984012604


In [86]:
print(good_companies)

['VTE', 'CMG', 'ELC', 'FPT', 'HIG', 'HPT', 'ITD', 'KST', 'LTC', 'ONE', 'PMT', 'POT', 'SAM', 'SGT', 'SRA', 'SRB', 'ST8', 'TST', 'UNI', 'VEC', 'VLA']


In [100]:
def norm_X(X):
    min_feature = np.min(X)
    max_feature = np.max(X)
    X_norm = (X - min_feature) / (max_feature - min_feature)
    return X_norm, min_feature, max_feature

def denorm(y, min_feature, max_feature):
    return y * (max_feature - min_feature) + min_feature


In [106]:
company_names.head()

,Unnamed: 0,ticker,group_code,company_name,company_short_name
0,0,VVS,UpcomIndex,Công ty Cổ phần Đầu tư Phát triển Máy Việt Nam,Đầu tư Phát triển Máy Việt Nam
1,1,XDC,UpcomIndex,Công ty TNHH MTV Xây dựng Công trình Tân Cảng,Xây dựng Công trình Tân Cảng
2,2,HSV,UpcomIndex,Công ty Cổ phần Gang Thép Hà Nội,Gang Thép Hà Nội
3,3,CST,UpcomIndex,Công ty Cổ phần Than Cao Sơn - TKV,Than Cao Sơn - TKV
4,4,BVL,UpcomIndex,Công ty Cổ phần BV Land,BV Land


In [147]:
#prediction for the next 7 days
portfolio = {}

def predict_company(company): 
    result = []
    short_name = str(company_names[company_names.ticker == company]['company_name'].values[0])
    group_code = str(company_names[company_names.ticker == company]['group_code'].values[0])
    filename = history_path + company + "-" + group_code + '-History.csv'
    X = pd.read_csv(filename).iloc[-window_size:][features]
    indicators = calc_indicators(X)
    X_np  = X.to_numpy().reshape(1, X.shape[0], X.shape[1])
    X_norm, min_feature, max_feature = norm_X(X_np)
    model = tf.keras.models.load_model('saved_model/vnstock/VTE', compile=False)
    y_pred = model.predict(X_norm)
    y_pred_denorm = denorm(y_pred, min_feature, max_feature)
    rsi = calc_indicator(X)['RSI'].iloc[-pred_time:]
    buy = np.argmin(rsi)
    sell = np.argmax(rsi)
    should_buy = buy if rsi.iloc[buy] < 20 else ""
    should_sell = sell if rsi.iloc[sell] > 80 else ""
    portfolio[company] = [company, short_name, y_pred_denorm.flatten(), should_buy, should_sell]

In [149]:
#Predict for the last 7 days
for company in good_companies: 
    predict_company(company)
    

1/1 [==============================] - 2s 2s/step


In [152]:
my_portfolio = pd.DataFrame.from_dict(portfolio, orient='index',
                       columns=['Ticker', 'Company Name', '7 Day Prediction', "Buy On", 'Sell On'])

In [153]:
my_portfolio

,Ticker,Company Name,7 Day Prediction,Buy On,Sell On
VTE,VTE,Công ty Cổ phần VINACAP Kim Long,"[6705.3076, 6703.3096, 6699.3613, 6694.137, 66...",,0
CMG,CMG,Công ty Cổ phần Tập đoàn Công nghệ CMC,"[41622.043, 41593.742, 41472.098, 41286.81, 41...",2,
ELC,ELC,Công ty Cổ phần Công nghệ - Viễn thông ELCOM,"[12343.875, 12468.934, 12556.421, 12538.179, 1...",5,0
FPT,FPT,Công ty Cổ phần FPT,"[81208.48, 81451.61, 81633.22, 81693.68, 81654...",5,0
HIG,HIG,Công ty Cổ phần Tập Đoàn HIPT,"[7077.114, 7070.3096, 7058.0767, 7079.3755, 71...",,
HPT,HPT,Công ty Cổ phần Dịch vụ Công nghệ Tin học HPT,"[14769.451, 14445.904, 14020.525, 13616.316, 1...",2,
ITD,ITD,Công ty Cổ phần Công nghệ Tiên Phong,"[12543.264, 12512.816, 12474.336, 12428.213, 1...",6,
KST,KST,Công ty Cổ phần KASATI,"[14564.437, 14688.747, 14797.672, 14889.408, 1...",,1
LTC,LTC,Công ty Cổ phần Điện nhẹ Viễn Thông,"[1120.9019, 1121.0873, 1121.2756, 1121.4598, 1...",0,5
ONE,ONE,Công ty Cổ phần Truyền thông số 1,"[5691.53, 5668.791, 5689.1997, 5720.899, 5743....",,


In [145]:
my_portfolio.to_csv("my_portfolio.csv")